# CF checker

A short illustration of what is currently checked by the CF (Climate and Forecast) conventions checker implemented in Valenspy.

In [3]:
import valenspy as vp
import xarray as xr
from pathlib import Path

## What is checked?

In Valenspy, the self implemented CF conventions checker checks the following:

- The global attributes `history` and `Conventions` are present.
- Per variable the attributes `standard_name`, `long_name` & `units` are present.
- Checks if the file is a netCDF file.

In addition the cf_checker compares the variables in the file with the CORDEX_variables.yml file. 
If the variable is in the CORDEX_variables.yml file, the checker will check if the variable has the correct attributes.

## Some examples
### CF compliant file
Starting with a netCDF compliant file.
Note that the is_cf_compliant function accepts a file path or an xarray dataset.

In [4]:
print(vp.demo_data_CF)
ds =  xr.open_dataset(vp.demo_data_CF)
ds

/dodrio/scratch/projects/2022_200/project_output/RMIB-UGent/vsc46032_kobe/ValEnsPy/src/valenspy/datafiles/tas_Amon_EC-Earth3-Veg_historical_r1i1p1f1_gr_195301-195312.nc


<xarray.Dataset> Size: 6MB
Dimensions:    (time: 12, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) datetime64[ns] 96B 1953-01-16T12:00:00 ... 1953-12-16T1...
  * lat        (lat) float64 2kB -89.46 -88.77 -88.07 ... 88.07 88.77 89.46
  * lon        (lon) float64 4kB 0.0 0.7031 1.406 2.109 ... 357.9 358.6 359.3
    height     float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] 192B ...
    lat_bnds   (lat, bnds) float64 4kB ...
    lon_bnds   (lon, bnds) float64 8kB ...
    tas        (time, lat, lon) float32 6MB ...
Attributes: (12/46)
    Conventions:                        CF-1.7 CMIP-6.2
    activity_id:                        CMIP
    branch_method:                      standard
    branch_time_in_child:               0.0
    branch_time_in_parent:              29219.0
    contact:                            cmip6-data@ec-earth.org
    ...                                 ...
    variant_label:                      r1i1p1f1
    license:                            CMIP6 model data produced by EC-Earth...
    cmor_version:                       3.4.0
    tracking_id:                        hdl:21.14100/18af2970-6a17-45fe-b629-...
    history:                            2019-06-06T07:27:13Z ; CMOR rewrote d...
    latest_applied_cmor_fixer_version:  v3.0

In [5]:
vp.cf_checks.is_cf_compliant(vp.demo_data_CF)

True

In [6]:
vp.cf_checks.is_cf_compliant(ds)

True

### Non-CF compliant file
The next examples include files which are not (completely) CF compliant.


In [7]:
vp.cf_checks.is_cf_compliant('file_that_does_not_exist.nc') #Any file which is not a netCDF file (no .nc extension and cannot be loaded as an xarray) will return False - even if it does not exist

False

E-OBS temperature file data set. 

In [8]:

eobs = xr.open_dataset(Path("/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc"), chunks={'time': 100})
eobs

<xarray.Dataset> Size: 35GB
Dimensions:    (latitude: 465, longitude: 705, time: 27028)
Coordinates:
  * latitude   (latitude) float64 4kB 25.05 25.15 25.25 ... 71.25 71.35 71.45
  * longitude  (longitude) float64 6kB -24.95 -24.85 -24.75 ... 45.35 45.45
  * time       (time) datetime64[ns] 216kB 1950-01-01 1950-01-02 ... 2023-12-31
Data variables:
    tg         (time, latitude, longitude) float32 35GB dask.array<chunksize=(100, 465, 705), meta=np.ndarray>
Attributes:
    E-OBS_version:  29.0e
    Conventions:    CF-1.4
    References:     http://surfobs.climate.copernicus.eu/dataaccess/access_eo...
    history:        Fri Mar 22 09:55:59 2024: ncks --no-abc -d time,0,27027 /...
    NCO:            netCDF Operators version 5.1.4 (Homepage = http://nco.sf....

In [9]:
vp.cf_checks.is_cf_compliant(Path("/dodrio/scratch/projects/2022_200/project_input/External/observations/EOBS/0.1deg/tg_ens_mean_0.1deg_reg_v29.0e.nc"))

True

It seems that E-OBS files is also CF compliant! However, currently the two datasets both contain temperature data but are not comparable.
E-OBS data set passed the is_cf_compliant function becuase tg is not in the CORDEX_variables.yml file. To be able to use the two datasets together, E-OBS data should be translated to the CORDEX standard.

In [10]:
eobs = eobs.rename({'tg': 'tas'})
eobs['tas'].attrs

{'units': 'Celsius',
 'long_name': 'mean temperature',
 'standard_name': 'air_temperature',
 'cell_methods': 'time: mean'}

In [11]:
vp.cf_checks.is_cf_compliant(eobs)

False

Now it is no longer "CF compliant" because the variable `tas` is expected to be in Kelvin, not in Celsius and some attributes are missing.

In [19]:
#Convert to Kelvin
eobs['tas'] = eobs['tas'] + 273.15
eobs['tas'].attrs['units'] = 'K'
eobs['tas'].attrs['long_name'] = "Near-Surface Air Temperature"
eobs['tas'].attrs['standard_name'] = "air_temperature"
eobs['tas'].attrs

{'units': 'K',
 'long_name': 'Near-Surface Air Temperature',
 'standard_name': 'air_temperature'}

In [21]:
vp.cf_checks.is_cf_compliant(eobs) #Now the data is CF compliant and compatible for direct usage with valenspy.demo_data_CF

True

### TODO

Once the E-OBS data convertor is implemented, show E-OBS conversions results in CF-compliant - CORDEX-compliant data.